# Yahoo Finance 
- March 2025: Y-Finance app access barred so have had to switch to taking price data from Yahoo Watchlists
- Watchlists split into three accounts (pd.read_html only reads a limited amount of entries at a time)
- First account: finlaygordonaugust2020@yahoo.com; August2020 (ETF prices)
- Second account: finlaygordonmarch2025@yahoo.com; March_2025 (ETF prices)
- Third account: finlaygordonapril2025@yahoo.com; April_2025 (index % changes)

# 1/ Access first account, extract watchlist

In [153]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from io import StringIO

options = Options()
#options.add_argument("--headless")  # Run browser in the background
driver = webdriver.Chrome(options=options)

url = "https://login.yahoo.com/"
driver.get(url)
time.sleep(2)

# Enter username
username_field = driver.find_element(By.ID, "login-username")
username_field.send_keys("finlaygordonaugust2020@yahoo.com")  # Replace with your username
username_field.send_keys(Keys.RETURN)
time.sleep(2)

# Enter password
password_field = driver.find_element(By.ID, "login-passwd")
password_field.send_keys("August2020")  # Replace with your password
password_field.send_keys(Keys.RETURN)
time.sleep(5)

# Navigate to the watchlist
watchlist_url = "https://finance.yahoo.com/portfolio/p_0/view/v1"
driver.get(watchlist_url)
time.sleep(5)

html_content = driver.page_source
html_io = StringIO(html_content)
tables = pd.read_html(html_io)
df = tables[0]

driver.quit()

# 2/ Access second account, extract watchlist

In [154]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from io import StringIO

options = Options()
#options.add_argument("--headless")  # Run browser in the background
driver = webdriver.Chrome(options=options)

url = "https://login.yahoo.com/"
driver.get(url)
time.sleep(2)

# Enter username
username_field = driver.find_element(By.ID, "login-username")
username_field.send_keys("finlaygordonmarch2025@yahoo.com")  # Replace with your username
username_field.send_keys(Keys.RETURN)
time.sleep(2)

# Enter password
password_field = driver.find_element(By.ID, "login-passwd")
password_field.send_keys("March_2025")  # Replace with your password
password_field.send_keys(Keys.RETURN)
time.sleep(5)

# Navigate to the watchlist
watchlist_url = "https://finance.yahoo.com/portfolio/p_0/view/v1"
driver.get(watchlist_url)
time.sleep(5)

html_content = driver.page_source
html_io = StringIO(html_content)
tables = pd.read_html(html_io)
df1 = tables[0]

driver.quit()

# 3/ Access third account, extract watchlist
- This is just index data

In [155]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from io import StringIO

options = Options()
#options.add_argument("--headless")  # Run browser in the background
driver = webdriver.Chrome(options=options)

url = "https://login.yahoo.com/"
driver.get(url)
time.sleep(2)

# Enter username
username_field = driver.find_element(By.ID, "login-username")
username_field.send_keys("finlaygordonapril2025@yahoo.com")  # Replace with your username
username_field.send_keys(Keys.RETURN)
time.sleep(2)

# Enter password
password_field = driver.find_element(By.ID, "login-passwd")
password_field.send_keys("April_2025")  # Replace with your password
password_field.send_keys(Keys.RETURN)
time.sleep(5)

# Navigate to the watchlist
watchlist_url = "https://finance.yahoo.com/portfolio/p_0/view/v1"
driver.get(watchlist_url)
time.sleep(5)

html_content = driver.page_source
html_io = StringIO(html_content)
tables = pd.read_html(html_io)
df1a = tables[0]

driver.quit()

# 3/ Create combined df
- Just for security prices contained in df and df1

In [156]:
df2 = pd.concat([df, df1])

# 4/ Match prices with ticker list

In [157]:
result = {}

ticker_list = ['IUSP.L','CNX1.L', 'EIMU.L', 'ERND.L', 'ERNS.L', 'EUE.L', 'EUNN.DE', 'FCSS.L', 'OMXSL.XC','XDN0.L', 'GSG', 'GSPX.L', 'IASP.L', 'IBC9.DE', 'IBTG.L', 'IDJG.L', 'IIND.L', 'IQQA.DE', 'IQQG.DE', 'IQQP.DE', 'ISF.L', 'IUKP.L', 'IUMO.L', 'IUSA.L', 'JCGI.L', 'LQDS.L', 'MCHI', 'NDIA.L', 'SGLN.L', 'SJPA.L', 'SXR8.DE', 'TI5G.L', 'VEMT.L', 'VERX.L', 'VFEM.L', 'VGEU.DE', 'VHYL.L', 'VJPN.L', 'VJPN.L', 'VMID.L', 'VUKE.L'] 

for ticker in ticker_list:

    if ticker in df2['Symbol'].values:

        price = df2.loc[df2['Symbol'] == ticker, 'Last Price'].values[0]
        currency = df2.loc[df2['Symbol'] == ticker, 'Currency'].values[0]
        result[ticker] = {'Last Price': price, 'Currency': currency}
    else:
         result[ticker] = {'Last Price': None, 'Currency': None}

df3 = pd.DataFrame.from_dict(result, orient='index').reset_index()
df3.columns = ['Symbol', 'Last Price', 'Currency']

# Watchlists do not include security name so import this from csv and merge with df3
df4 = pd.read_csv('C:\\Users\\piers\\OneDrive\\Piers work files\\Python files\\PB_portfolio\\Ticker_name_map.csv')

df3 = pd.merge(df3, df4, left_on='Symbol', right_on='ticker', how='left')
df3 = df3.drop(columns=['ticker'])
df3 = df3.drop_duplicates()


# 5/ Back to original code, substituting y-Finance with Watchlist price data

In [158]:
import pandas as pd
import pprint
from collections import defaultdict
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import smtplib
import openpyxl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

ticker_list = ['IUSP.L','CNX1.L', 'EIMU.L', 'ERND.L', 'ERNS.L', 'EUE.L', 'EUNN.DE', 'FCSS.L', 'OMXSL.XC','XDN0.L', 'GSG', 'GSPX.L', 'IASP.L', 'IBC9.DE', 'IBTG.L', 'IDJG.L', 'IIND.L', 'IQQA.DE', 'IQQG.DE', 'IQQP.DE', 'ISF.L', 'IUKP.L', 'IUMO.L', 'IUSA.L', 'JCGI.L', 'LQDS.L', 'MCHI', 'NDIA.L', 'SGLN.L', 'SJPA.L', 'SXR8.DE', 'TI5G.L', 'VEMT.L', 'VERX.L', 'VFEM.L', 'VGEU.DE', 'VHYL.L', 'VJPN.L', 'VJPN.L', 'VMID.L', 'VUKE.L'] 
curncy_list = ['EURGBP=X', 'GBP=X']

short_names = []
bid_prices = []
previous_prices = []
currency = []

fx_rates = []
long_names = []


for ticker in ticker_list:
    # Check if the ticker exists in df3
    if ticker in df3['Symbol'].values:
        # Extract corresponding "name", "Last Price" and "Currency" from df3
        short_name = df3.loc[df3['Symbol'] == ticker, 'name'].values[0]
        short_names.append(short_name)
        last_price = df3.loc[df3['Symbol'] == ticker, 'Last Price'].values[0]
        bid_prices.append(last_price)
        crncy = df3.loc[df3['Symbol'] == ticker, 'Currency'].values[0]
        currency.append(crncy)
        # Append values to the lists
        
          # Use Last Price as bid_price
        
    else:
        # If ticker is not found in df3, append None or a placeholder
        short_names.append(None)
        bid_prices.append(None)
        currency.append(None)

for ticker in curncy_list:
    if ticker in df['Symbol'].values:
        rate = df.loc[df['Symbol'] == ticker, 'Last Price'].values[0]
        fx_rates.append(rate)
        name = df.loc[df['Symbol'] == ticker, 'Symbol'].values[0]
        long_names.append(name)

    else:
        # If ticker is not found in df3, append None or a placeholder
        fx_rates.append(None)
        long_names.append(None)
    

price_dict ={}
for ticker_list, bid_prices, short_names, currency in zip(ticker_list, bid_prices, short_names, currency):
    price_dict[ticker_list] = {'name': short_names, 'price': bid_prices, 'currency': currency}

fx_rate_dict ={}
for curncy_list, fx_rates, long_names in zip(curncy_list, fx_rates, long_names):
    fx_rate_dict[curncy_list] = {'name': long_names, 'rate': fx_rates}

# GETTING PORTFOLIO DATA FROM EXCEL, STORING IN A DICTIONARY

df5 = pd.read_excel('C:\\Users\\piers\\OneDrive\\Piers work files\\Python files\\PB_portfolio\\Holdings_data.xlsx')
columns = ['Instrument', 'Ticker_second', 'Account', 'Cur.', 'FX rate', 'Holding']
df5[columns]

instruments = df5['Instrument'].tolist()
tickers = df5['Ticker_second'].tolist()
accounts = df5['Account'].tolist()
holdings = df5['Holding'].tolist()


data = {}

for ticker, account, holding in zip(tickers, accounts, holdings):
    if ticker not in data:
        data[ticker] = []
    data[ticker].append({'account': account, 'holding': holding})

for key, entries in data.items():
    if key in price_dict:
        price = price_dict[key]['price']
        name = price_dict[key]['name']
        currency = price_dict[key]['currency']        
        for entry in entries:
            entry['price'] = price
            entry['name'] = name
            entry['currency'] = currency
            if currency == 'GBp':
                entry['fx'] = .01
            elif currency == 'GBP':
                entry['fx'] = 1
            elif currency == 'EUR':
                entry['fx'] = fx_rate_dict['EURGBP=X']['rate']
            elif currency == 'USD':
                entry['fx'] = fx_rate_dict['GBP=X']['rate']
            entry['holding'] = int(entry['holding'])
            entry['value'] = entry['price'] * entry['holding'] * entry['fx']
    elif key == 'Cash':
        for entry in entries:
            entry['name'] = 'Cash'
            entry['currency'] = 'GBP'
            entry['holding'] = int(entry['holding'])
            entry['fx'] = 1
            entry['value'] = entry['holding'] * entry['fx']
    elif key == 'Non-traded':
        for entry in entries:
            entry['name'] = 'Non-traded'    
            entry['currency'] = 'GBP'
            entry['fx'] = 1
            entry['holding'] = int(entry['holding'])
            entry['value'] = entry['holding'] * entry['fx']
    elif key == 'VASSTAI.L':
        for entry in entries:
            entry['name'] = 'VANGUARD STERLING SHORT-TERM MM'    
            entry['currency'] = 'GBP'
            entry['fx'] = 1
            entry['holding'] = int(entry['holding'])
            entry['value'] = entry['holding']
    elif key == 'SX7PEX.DE':
        for entry in entries:
            entry['name'] = 'ISHARES STOXX EUROPE 600 BANKS'    
            entry['currency'] = 'EUR'
            entry['fx'] = fx_rate_dict['EURGBP=X']['rate']
            entry['holding'] = int(entry['holding'])
            entry['value'] = 25.77 * entry['holding'] * entry['fx']
    elif key == 'SCHINSA.L':
        for entry in entries:
            entry['name'] = 'SCHRODER MOORGATE I FUND'    
            entry['currency'] = 'GBp'
            entry['fx'] = .01
            entry['holding'] = int(entry['holding'])
            entry['value'] = 97.01 * entry['holding'] * entry['fx']

# Calculating total values for all items and summing per geography

key_list = list(data.keys())
values_dict = {}
for key in key_list:
    values_dict[key] = sum(entry['value'] for entry in data[key])

account_dict = {}
for key in key_list:
    account_dict[key] = [entry['account'] for entry in data[key]]


USA_tickers = ['IUSP.L','IUMO.L','GSPX.L','SXR8.DE','IUSA.L','CNX1.L']
USA = sum(values_dict[key] for key in USA_tickers)
Europe_tickers = ['IDJG.L','VERX.L','IQQG.DE','IQQA.DE','VGEU.DE','EUE.L','IQQP.DE','SX7PEX.DE']
Europe = sum(values_dict[key] for key in Europe_tickers)
Nordic_tickers = ['OMXSL.XC', 'XDN0.L']
Nordic = sum(values_dict[key] for key in Nordic_tickers)    
UK_tickers = ['VUKE.L','VMID.L','VMID.L','VMID.L','ISF.L','SCHINSA.L','IUKP.L']
UK = sum(values_dict[key] for key in UK_tickers)
Japan_tickers = ['SJPA.L','EUNN.DE','VJPN.L']
Japan = sum(values_dict[key] for key in Japan_tickers)
EM_tickers = ['VFEM.L', 'EIMU.L', 'IASP.L']
EM = sum(values_dict[key] for key in EM_tickers)
China_tickers = ['FCSS.L', 'JCGI.L','MCHI']
China = sum(values_dict[key] for key in China_tickers)
India_tickers = ['NDIA.L', 'IIND.L']
India = sum(values_dict[key] for key in India_tickers)
Yield_tickers = ['VHYL.L']
Yield = sum(values_dict[key] for key in Yield_tickers)
FixedIncome_tickers = ['VEMT.L','TI5G.L','ERND.L','IBTG.L','IBC9.DE','LQDS.L', 'ERNS.L']
Fixed_income = sum(values_dict[key] for key in FixedIncome_tickers)
MM_tickers = ['VASSTAI.L']
MM = sum(values_dict[key] for key in MM_tickers)
Commodity_tickers = ['SGLN.L', 'GSG']
Commodities = sum(values_dict[key] for key in Commodity_tickers)
NonTraded_tickers = ['Non-traded']
Non_traded = sum(values_dict[key] for key in NonTraded_tickers)
Cash_tickers = ['Cash']
Cash = sum(values_dict[key] for key in Cash_tickers)
Total = (USA+Europe+Nordic+UK+Japan+EM+China+India+Yield+Fixed_income+MM+Commodities+Non_traded+Cash)

today = datetime.today().strftime('%Y-%m-%d')
df6=pd.DataFrame({'USA':USA, 'Europe':Europe, 'Nordic': Nordic, 'UK':UK,'Japan':Japan,'EM':EM,'China':China,'India':India,'Yield':Yield,'Fixed income':Fixed_income, 'Money market':MM,'Commodities':Commodities, 'Non-traded':Non_traded, 'Cash':Cash, 'Total':Total }, index = [today])
pd.options.display.float_format = '{:,.0f}'.format
df6T = df6.T
df6_html = df6T.to_html(index=True)


# Create an empty list to store DataFrames
dfs = []

for key, value in data.items():
    temp_df = pd.DataFrame(value)
    temp_df['ticker'] = key
    dfs.append(temp_df)

df7 = pd.concat(dfs, ignore_index=True)
df7 = df7[['ticker', 'account', 'name', 'currency', 'fx', 'holding', 'price', 'value']]
"""
# IMPORTING PREVIOUS DAY'S VALUES FROM EXCEL

excel_file_path = 'C:\\Users\\piers\\OneDrive\\Piers work files\\Python files\\PB_portfolio\\Summary_history2.xlsx'
sheet_name = 'Sheet1'
previous_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
df_previous = pd.read_excel(excel_file_path, sheet_name=previous_date, skiprows=[1], index_col=None)
df_previous.set_index(df_previous.columns[0], inplace=True)
df_previous.index.name = ''

df2 = pd.concat([df_previous, df1])
df_pct_chge = (df2.pct_change()*100).map(lambda x: '{:,.2f}'.format(x))
df2_pct_chge = pd.concat([df2, df_pct_chge.drop(df_pct_chge.index[0])])
df2_pct_chgeT = df2_pct_chge.T
df2_pct_chge_html = df2_pct_chgeT.to_html(index=True)
"""
# CREATING DF WITH VALUES PER ACCOUNT

from collections import defaultdict

account_values = defaultdict(list)
for key, entries in data.items():
    for entry in entries:        
        account_name = entry['account']        
        account_values[account_name].append(entry['value'])
account_values_dict = dict(account_values)
sums_dict = {}
for key, values in account_values_dict.items():    
    total_value = sum(values)
    total_value = '{:,.0f}'.format(total_value)    
    sums_dict[key] = total_value

df8 = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['Value'])
df8['Value'] = df8['Value'].str.replace(',', '').astype(float)
df8_sorted = df8.sort_values(by='Value', ascending=False)
df8_sorted_html = df8_sorted.to_html(index=True)

# RETRIEVE INDEX DATA

tickerStrings = ['^GSPC', '^FTAS', '^GDAXI', '^FCHI', '^N225', '^HSI', 'HSBA.L', 'GBP=X', 'GBPEUR=X']
name_list = {'^GSPC': 'S&P500', '^FTAS':'FTSE AllSh', '^GDAXI':'DAX', '^FCHI':'CAC40', '^N225':'Nikkei 225', '^HSI':'Hang Seng', 'HSBA.L':'HSBC','GBP=X':'GBPUSD', 'GBPEUR=X':'GBPEUR'}

df9 = df1a[['Symbol', 'Last Price', 'Change (%)']].loc[0:8]
df9['Name'] = df9['Symbol'].map(name_list)
df9_html = df9.to_html(index=True)


# RECORD IN EXCEL (df7 AND df6)
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
workbook = load_workbook('C:\\Users\\piers\\OneDrive\\Piers work files\\Python files\\PB_portfolio\\Portfolio_history3.xlsx')
new_sheet = workbook.create_sheet(title=today)
for row in dataframe_to_rows(df7, index=True, header=True):
    new_sheet.append(row)
workbook.save('C:\\Users\\piers\\OneDrive\\Piers work files\\Python files\\PB_portfolio\\Portfolio_history3.xlsx')

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
workbook = load_workbook('C:\\Users\\piers\\OneDrive\\Piers work files\\Python files\\PB_portfolio\\Summary_history2.xlsx')
new_sheet = workbook.create_sheet(title=today)
for row in dataframe_to_rows(df6, index=True, header=True):
    new_sheet.append(row)
workbook.save('C:\\Users\\piers\\OneDrive\\Piers work files\\Python files\\PB_portfolio\\Summary_history2.xlsx')


# SEND EMAIL
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

subject = "PB Portfolio"
body = ""
#body = f"{sum_value_formatted}\n\n{timestamp}"
sender = "finlaygordonmarch2024@gmail.com"
recipients = ["pierslbrown@gmail.com"]
password = "pnlp kdpr blse fuwo"
attachment_path = ""
filename = ""


def send_email(subject, body, sender, recipients, password, attachment_path, filename):    
    msg = MIMEMultipart()    
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)

    # Add the text message
    msg.attach(MIMEText(body, 'plain'))   
    # msg.attach(MIMEText(df2_pct_chge_html, "html"))
    msg.attach(MIMEText(df6_html, "html"))
    additional_text = "Non-traded = HSBC pension + HSBC shares + KB pension (Aegon, Scottish Widows)"
    msg.attach(MIMEText(additional_text, "html"))
    msg.attach(MIMEText(df8_sorted_html, "html"))
    msg.attach(MIMEText(df9_html, "html"))
    
    # Send the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
       smtp_server.login(sender, password)
       smtp_server.sendmail(sender, recipients, msg.as_string())
    print("Executed successfully. Email sent!")

send_email(subject, body, sender, recipients, password, attachment_path, filename)




Executed successfully. Email sent!
